# Analysis of Wind Data in Ireland

This notebook looks at weather data related to wind in Ireland.
The data was taken from Met Éireann and is used here to explore how wind speed changes over time and what that might mean for wind energy.

## About the Data

Before analysing the data, it is useful to understand where it comes from and what information it contains.

## Preparing the Dataset

The dataset contains some missing and inconsistent values which need to be fixed before doing any analysis.

## Exploring Wind Speed

In this section, different summaries and plots are used to explore wind speed patterns over time.

## Discussion of Results

This section discusses what was observed from the analysis and how it may relate to wind energy.